# Company Scores (nbdev)

Company scoring facades and tests here. Export with `#| export` cells.


In [ ]:
#| default_export company_scores
#| export
import logging
from datetime import datetime, date
from typing import List, Dict, Optional

logger = logging.getLogger(__name__)

class CompanyScoreCalculator:
    def __init__(self, supabase_client):
        self.supabase = supabase_client

    async def calculate_all_company_scores(self, academic_year: int, calculation_date: Optional[date] = None) -> Dict[str, any]:
        if calculation_date is None:
            from datetime import date as _d
            calculation_date = _d.today()
        logger.info(f"Starting company score calculation for academic year {academic_year}")
        logger.info(f"Calculation date: {calculation_date}")
        start_time = datetime.now()
        results = {
            'academic_year': academic_year,
            'calculation_date': calculation_date.isoformat(),
            'companies_processed': 0,
            'total_companies': 0,
            'phases': [],
            'total_execution_time': None,
            'status': 'in_progress'
        }
        companies_response = await self.supabase.table('companies').select('id, name').eq('is_active', True).execute()
        companies = companies_response.data
        if not companies:
            return {'companies_processed': 0, 'error': 'No active companies found'}
        results['total_companies'] = len(companies)
        # Subcategory
        phase_start = datetime.now()
        for company in companies:
            await self.supabase.rpc('calculate_company_subcategory_scores', {
                'p_company_id': company['id'],
                'p_academic_year_start': academic_year,
                'p_calculation_date': calculation_date.isoformat()
            }).execute()
        results['phases'].append({
            'phase': 'Calculate Company Subcategory Scores',
            'companies_processed': len(companies),
            'execution_time_seconds': (datetime.now() - phase_start).total_seconds(),
            'status': 'completed'
        })
        # Category
        phase_start = datetime.now()
        for company in companies:
            await self.supabase.rpc('calculate_company_category_scores', {
                'p_company_id': company['id'],
                'p_academic_year_start': academic_year,
                'p_calculation_date': calculation_date.isoformat()
            }).execute()
        results['phases'].append({
            'phase': 'Calculate Company Category Scores',
            'companies_processed': len(companies),
            'execution_time_seconds': (datetime.now() - phase_start).total_seconds(),
            'status': 'completed'
        })
        # Holistic
        phase_start = datetime.now()
        for company in companies:
            await self.supabase.rpc('calculate_company_holistic_gpa', {
                'p_company_id': company['id'],
                'p_academic_year_start': academic_year,
                'p_calculation_date': calculation_date.isoformat()
            }).execute()
        results['phases'].append({
            'phase': 'Calculate Company Holistic GPAs',
            'companies_processed': len(companies),
            'execution_time_seconds': (datetime.now() - phase_start).total_seconds(),
            'status': 'completed'
        })
        results['total_execution_time'] = (datetime.now() - start_time).total_seconds()
        results['status'] = 'completed'
        results['companies_processed'] = len(companies)
        return results

    async def get_company_rankings(self, academic_year: int, calculation_date: Optional[date] = None) -> List[Dict]:
        if calculation_date is None:
            from datetime import date as _d
            calculation_date = _d.today()
        response = await self.supabase.table('company_holistic_gpa').select(
            'company_id, holistic_gpa, category_breakdown, calculation_date'
        ).eq('academic_year_start', academic_year).eq(
            'calculation_date', calculation_date.isoformat()
        ).order('holistic_gpa', desc=True).execute()
        rankings = []
        for i, record in enumerate(response.data, 1):
            company_response = await self.supabase.table('companies').select('name').eq('id', record['company_id']).execute()
            company_name = company_response.data[0]['name'] if company_response.data else 'Unknown Company'
            rankings.append({
                'rank': i,
                'company_id': record['company_id'],
                'company_name': company_name,
                'holistic_gpa': float(record['holistic_gpa']),
                'category_breakdown': record['category_breakdown'],
                'calculation_date': record['calculation_date']
            })
        return rankings

    async def validate_company_scores(self, academic_year: int, calculation_date: Optional[date] = None) -> Dict:
        if calculation_date is None:
            from datetime import date as _d
            calculation_date = _d.today()
        validation_results = {
            'academic_year': academic_year,
            'calculation_date': calculation_date.isoformat(),
            'validation_checks': [],
            'status': 'passed'
        }
        companies_response = await self.supabase.table('companies').select('id, name').eq('is_active', True).execute()
        companies = companies_response.data
        holistic_gpas_response = await self.supabase.table('company_holistic_gpa').select('company_id').eq(
            'academic_year_start', academic_year
        ).eq('calculation_date', calculation_date.isoformat()).execute()
        companies_with_gpas = {gpa['company_id'] for gpa in holistic_gpas_response.data}
        missing_gpas = [c['name'] for c in companies if c['id'] not in companies_with_gpas]
        if missing_gpas:
            validation_results['validation_checks'].append({'check': 'All companies have holistic GPAs', 'status': 'failed', 'details': f"Missing GPAs for companies: {missing_gpas}"})
            validation_results['status'] = 'failed'
        else:
            validation_results['validation_checks'].append({'check': 'All companies have holistic GPAs', 'status': 'passed', 'details': f"All {len(companies)} companies have holistic GPAs"})
        return validation_results
